# Erase Schema
This notebook drops all user-defined functions, tables, and views from a schema and then removes the schema. Provide the catalog and schema names with the widgets below and run all cells.

In [ ]:
try:
    dbutils.widgets.text("1.catalog", "catalog")
    dbutils.widgets.text("2.schema", "schema")
    catalog = dbutils.widgets.get("1.catalog")
    schema = dbutils.widgets.get("2.schema")
except NameError:
    catalog = "catalog"
    schema = "schema"

In [ ]:
spark.sql(f"USE CATALOG `{catalog}`")

skip_tables = set()
skip_views = set()

# Drop functions sequentially
funcs_df = spark.sql(f"SHOW USER FUNCTIONS IN `{catalog}`.`{schema}`")
for f in funcs_df.collect():
    func_name = f[0]
    spark.sql(f"DROP FUNCTION IF EXISTS `{catalog}`.`{schema}`.`{func_name}`")

# Drop tables sequentially
tables_df = spark.sql(f"SHOW TABLES IN `{catalog}`.`{schema}`")
for t in tables_df.collect():
    desc = spark.sql(f"DESCRIBE EXTENDED `{catalog}`.`{schema}`.`{t.tableName}`").collect()
    t_type = next((row.data_type for row in desc if str(row.col_name).upper() == "TYPE"), "").upper().replace("_", " ")
    if t_type == "VIEW":
        spark.sql(f"DROP VIEW IF EXISTS `{catalog}`.`{schema}`.`{t.tableName}`")
    elif t_type == "MATERIALIZED_VIEW":
        spark.sql(f"DROP MATERIALIZED VIEW IF EXISTS `{catalog}`.`{schema}`.`{t.tableName}`")
    else:
        spark.sql(f"DROP TABLE IF EXISTS `{catalog}`.`{schema}`.`{t.tableName}`")

spark.sql(f"DROP SCHEMA IF EXISTS `{catalog}`.`{schema}`")

In [ ]:
print(f"Schema `{schema}` in catalog `{catalog}` cleanup complete.")